In [1]:
%load_ext autoreload
%autoreload 2

# testing the handler

In [2]:
from retrieva.handler import RagHandler
from dotenv import load_dotenv
import os
from retrieva import ROOT_PATH
from retrieva.data import DATA_PATH
# used in dev; in production pass the env variable to the containers
load_dotenv(os.path.join(ROOT_PATH, ".env"))

True

In [3]:
from IPython.display import Markdown, display, clear_output

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [12]:
rag_handler = RagHandler(
    index_name="SageMakerDocs",
    weaviate_url=os.environ["WEAVIATE_URL"],
    data_path=DATA_PATH
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [17]:
# testing
query = "how can I solve a RL aws problem with sagemaker?"
# query = "am I an OVNI?"
resp_stream = rag_handler.user_prompt_streaming(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [18]:
sentence = ""
for text in resp_stream.response_gen:
    # return the texts as they arrive.
    clear_output()
    sentence += text
    print(sentence)

To solve a RL problem using Amazon SageMaker, you can follow these steps:

1. Formulate the RL problem by defining the components of the Markov Decision Process, such as the objective, environment, state, action, and reward.
2. Define the RL environment, which can be the real world or a simulation.
3. Define the presets to configure the RL training jobs and set hyperparameters for the RL algorithms.
4. Write the training code as a Python script and pass it to a SageMaker training job.
5. Train the RL model using the SageMaker `RLEstimator` in the Amazon SageMaker Python SDK.

For complete code examples and further reading, you can refer to the sample notebooks at https://github.com/awslabs/amazon-sagemaker-examples/tree/master/reinforcement-learning.


In [6]:
prompts_dict = rag_handler.engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:summary_template<br>**Text:** <br>

You are a summarization service to help users navigate proprietary documentation of their companies.
Always include the filepaths from the nodes in the context information to support the answer as well as further reading that may be mentioned in the docs. Remove '/home/peterdays/Documents/personal/retrieva/artifacts/sagemaker_documentation_small' from the metadatas' filepaths
Context information is below.
---------------------
{context_str}
---------------------
If the query and the context don't make sense answer with 'There is no good match in the docs for this prompt'
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>